In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import mylib

In [2]:
def text2binary(fichier):
    char=""
    file = open(fichier, 'r')
    while True:
        c=file.read(1)
        if not c: 
            break
        char += bin(ord(c))[2:].zfill(8)
    file.close()
    return char

In [3]:
def image2binary(image):
    global nl,nc
    
    I_gray=np.around(mylib.rgb2gray(image))
    nl,nc=I_gray.shape
    I=mylib.Img2liste_row(I_gray)
    
    return ''.join([bin(int(i))[2:].zfill(8) for i in I])

In [4]:
import wave

def audio2binary(audio): 
    global nchannels,sampwidth,framerate,nframes,comptype,compname
    (nchannels,sampwidth,framerate,nframes,comptype,compname) = audio.getparams()
    data=audio.readframes(-1)
    return ''.join([bin(i)[2:].zfill(8) for i in data])

In [5]:
#write data in texte file
def one(suit):
    liste=mylib.bin2dec(suit)
    return ''.join([chr(int(i,2)) for i in liste])

In [6]:
#write data in image file
def two(suit):
    
    liste=mylib.bin2dec(suit)
    
    #converts liste of 8 bits into pixels values
    pixels=[int(i,2) for i in liste]
        
    return mylib.liste2img_row(pixels,nl,nc)

In [7]:
import wave

#write data in audio file
def three(suit):
    
    liste=mylib.bin2dec(suit)
    
    ##converts liste of 8 bits into ascci values
    data_ori=[int(i,2) for i in liste]
    
    #the next 3 lines are for create audio file 
    path="Audio/conveted.wav"
    audio=wave.open(path,"w")
    audio.setparams((nchannels,sampwidth,framerate,nframes,comptype,compname))
    audio.writeframes(np.frombuffer(bytes(data_ori),np.dtype(int)))
    
    return path

In [8]:
def RZUni(seq,tb):
    return list(np.concatenate([[5]*(tb//2)+[0]*(tb//2) if i == "1" else [0]*tb  for i in seq]))

In [9]:
def decRZUni(l,tb):
    s=""
    for i in range(0,len(l),tb):
        if(l[i:i+tb//2]==[5]*(tb//2) and l[i+tb//2:i+tb]==[0]*(tb//2)):
            s+="1"
        elif(l[i:i+tb]==[0]*(tb)):
            s+="0"
        else:
            print("Bit erronné")
            break
    return s

In [10]:
def RZBi(seq,tb):
    return list(np.concatenate([[5]*(tb//2)+[0]*(tb//2) if i == "1" else [-5]*(tb//2)+[0]*(tb//2)  for i in seq]))

In [11]:
def decRZBi(l,tb):
    s=""
    for i in range(0,len(l),tb):
        if(l[i:i+tb//2]==[5]*(tb//2) and l[i+tb//2:i+tb]==[0]*(tb//2)):
            s+="1"
        elif(l[i:i+tb//2]==[-5]*(tb//2) and l[i+tb//2:i+tb]==[0]*(tb//2)):
            s+="0"
        else:
            print("Bit erronné")
            break
    return s

In [12]:
def NRZ(seq,tb):
    return list(np.concatenate([[5]*tb if i == "1" else [-5]*tb for i in seq]))

In [13]:
def decNRZ(l,tb):
    s=""
    for i in range(0,len(l),tb):
        if(l[i:i+tb]==[5]*(tb)):
            s+="1"
        elif(l[i:i+tb]==[-5]*(tb)):
            s+="0"
        else:
            print("Bit erronné")
            break
    return s

In [14]:
def NRZI(seq,tb):
    Lx=[]
    for i in range(len(seq)):
        if(i==0):
            if seq[i] == "1":
                Lx+=[5]*tb
            else:
                Lx+=[-5]*tb
        else:
            if seq[i] == "1":
                Lx+=[Lx[-1]]*tb
            else:
                Lx+=[-Lx[-1]]*tb
    return Lx

In [15]:
def decNRZI(l,tb):
    s=decNRZ(l[:tb],tb)
    for i in range(tb,len(l),tb):
        if(l[i:i+tb]==[l[i]]*(tb) and l[i]!=l[i-1]):
            s+="0"
        elif(l[i:i+tb]==[l[i]]*(tb) and l[i]==l[i-1]):
            s+="1"
        else:
            print("Bit erronné")
            break
    return s

In [16]:
def Manchester(seq,tb):
    return list(np.concatenate([[5]*(tb//2)+[-5]*(tb//2) if i == "1" else [-5]*(tb//2)+[5]*(tb//2) for i in seq]))

In [17]:
def decManchester(l,tb):
    s=""
    for i in range(0,len(l),tb):
        if(l[i:i+tb//2]==[5]*(tb//2) and l[i+tb//2:i+tb]==[-5]*(tb//2)):
            s+="1"
        elif(l[i:i+tb//2]==[-5]*(tb//2) and l[i+tb//2:i+tb]==[5]*(tb//2)):
            s+="0"
        else:
            print("Bit erronné")
            break
    return s

In [18]:
def ManchesterDiff(seq,tb):
    Lx=[]
    for i in range(len(seq)):
        if(i==0):
            if seq[i] == "1":
                Lx+=[5]*(tb//2)+[-5]*(tb//2)
            else:
                Lx+=[-5]*(tb//2)+[5]*(tb//2)
        else:
            if seq[i] == "1":
                Lx+=[Lx[-1]]*(tb//2)+[-Lx[-1]]*(tb//2)
            else:
                Lx+=[-Lx[-1]]*(tb//2)+[Lx[-1]]*(tb//2)
    return Lx

In [19]:
def decManchesterDiff(l,tb):
    s=decManchester(l[:tb],tb)
    for i in range(tb,len(l),tb):
        if(l[i:i+tb//2]==[l[i-1]]*(tb//2) and l[i+(tb//2):i+tb]==[-l[i-1]]*(tb//2)):
            s+="1"
        elif(l[i:i+tb//2]==[-l[i-1]]*(tb//2) and l[i+(tb//2):i+tb]==[l[i-1]]*(tb//2)):
            s+="0"
        else:
            print("Bit erronné")
            break
    return s

In [20]:
def Miller(seq,tb):
    Lx=[]
    for i in range(len(seq)):
        if(i==0):
            if seq[i] == "1":
                Lx+=[5]*(tb//2)+[-5]*(tb//2)
            else:
                Lx+=[-5]*tb
        else:
            if seq[i] == "1":
                Lx+=[Lx[-1]]*(tb//2)+[-Lx[-1]]*(tb//2)
            else:
                if(seq[i-1] == "1"):
                    Lx+=[Lx[-1]]*tb
                else:
                    Lx+=[-Lx[-1]]*tb
    return Lx

In [21]:
def decMiller(l,tb):
    s=""
    if(l[:tb//2]==[5]*(tb//2) and l[tb//2:tb]==[-5]*(tb//2)):
        s+="1"
    elif(l[:tb]==[-5]*(tb)):
        s+="0"
    else:
        print("Bit erronné")
        return s
    for i in range(tb,len(l),tb):
        if(l[i]!=l[i+tb//2]):
            if(l[i]==l[i-1] and l[i:i+tb//2]==[l[i]]*(tb//2) and l[i+tb//2:i+tb]==[l[i+tb//2]]*(tb//2)):
                s+="1"
            else:
                print("Bit erronné")
                break
        else:
            if(l[i:i+tb]==[l[i]]*(tb)):
                if((l[i]==l[i-1] and s[-1]=="1") or (l[i]!=l[i-1] and s[-1]=="0")):
                    s+="0"
            else:
                print("Bit erronné")
                break
    return s

In [22]:
def CMI(seq,tb):
    Lx=[]
    for i in range(len(seq)):
        if seq[i] == "1":
                Lx+=[-5]*(tb//2)+[5]*(tb//2)
        else:
            if(i==0):
                Lx=[-5]*tb
            elif(seq[i-1] == "1"):
                Lx+=[Lx[-1]]*tb
            elif(seq[i-1] == "0"):
                Lx+=[-Lx[-1]]*tb
    return Lx

In [23]:
def decCMI(l,tb):
    s=""
    if(l[:tb//2]==[-5]*(tb//2) and l[tb//2:tb]==[5]*(tb//2)):
        s+="1"
    elif(l[:tb]==[-5]*(tb)):
        s+="0"
    else:
        print("Bit erronné")
        return s
    for i in range(tb,len(l),tb):
        if(l[i:i+tb//2]==[-5]*(tb//2) and l[i+tb//2:i+tb]==[5]*(tb//2)):
            s+="1"
        elif(l[i:i+tb]==[l[i]]*(tb)):
            if((l[i]==l[i-1] and s[-1]=="1") or (l[i]!=l[i-1] and s[-1]=="0")):
                s+="0"
            else:
                print("Bit erronné")
                break
    return s

In [24]:
def AMI(seq,tb):
    Lx=[]
    for i in range(len(seq)):
        if(seq[i] == "0"):
            Lx+=[0]*tb
        else:
            if(i==seq.find("1")):
                Lx+=[5]*tb
                sw=-5
            else:
                Lx+=[sw]*tb
                sw=-sw
    return Lx

In [25]:
def decAMI(l,tb):
    s=""
    for i in range(0,len(l),tb):
        if(l[i:i+tb]==[0]*tb):
            s+="0"
        elif(i == 0):
            if(l[i:i+tb]==[5]*tb):
                s+="1"
                sw=-5
        elif(l[i:i+tb]==[sw]*tb):
            s+="1"
            sw=-sw
        else:
            print("Bit erronné")
            break
    return s

In [26]:
def MLT3(seq,tb):
    Lx=[]
    sw=[5,0,-5,0]
    j=4
    for i in range(len(seq)):
        if(seq[i] == "1"):
            if(i==seq.find("1")):
                Lx+=[5]*tb
            else:
                j=(j+1)%4
                Lx+=[sw[j]]*tb
        else:
            if(i==0):
                Lx+=[0]*tb
            else:
                Lx+=[Lx[-1]]*tb
    return Lx

In [27]:
def decMLT3(l,tb):
    s=""
    sw=[5,0,-5,0]
    j=4
    if(l[:tb]==[0]*tb):
        s+="0"
    elif(l[:tb]==[5]*tb):
        s+="1"
        j=0
    else:
        print("Bit erronné")
        return s
    for i in range(tb,len(l),tb):
        if(l[i:i+tb]==[l[i-1]]*tb):
            s+="0"
        elif(l[i:i+tb]==[sw[(j+1)%4]]*tb):
            s+="1"
            j=(j+1)%4
        else:
            print("Bit erronné")
            break
    return s

In [28]:
def TwoB1Q(seq,tb):
    Lx=[]
    for i in range(0,len(seq),2):
        if seq[i:i+2] == "10":
            Lx+=[3]*tb
        elif seq[i:i+2] == "11":
            Lx+=[1]*tb
        elif seq[i:i+2] == "01":
            Lx+=[-1]*tb
        elif seq[i:i+2] == "00":
            Lx+=[-3]*tb
    return Lx

In [29]:
def dec2B1Q(l,tb):
    s=""
    for i in range(0,len(l),tb):
        if(l[i:i+tb]==[3]*tb):
            s+="10"
        elif(l[i:i+tb]==[1]*tb):
            s+="11"
        elif(l[i:i+tb]==[-1]*tb):
            s+="01"
        elif(l[i:i+tb]==[-3]*tb):
            s+="00"
        else:
            print("Bit erronné")
            break
    return s

In [30]:
def signal_encode(signal,seq,tb):
    if signal=='RZ Uni':
        return RZUni(seq,tb)
    elif signal=='RZ Bi':
        return RZBi(seq,tb)
    elif signal=='NRZ':
        return NRZ(seq,tb)
    elif signal=='NRZI':
        return NRZI(seq,tb)
    elif signal=='Manchester':
        return Manchester(seq,tb)
    elif signal=='Manchester D':
        return ManchesterDiff(seq,tb)
    elif signal=='Miller':
        return Miller(seq,tb)
    elif signal=='CMI':
        return CMI(seq,tb)
    elif signal=='AMI':
        return AMI(seq,tb)
    elif signal=='MLT3':
        return MLT3(seq,tb)
    else:
        return TwoB1Q(seq,tb)

In [31]:
def signal_decode(signal,seq,tb):
    if signal=='RZ Uni':
        return decRZUni(seq,tb)
    elif signal=='RZ Bi':
        return decRZBi(seq,tb)
    elif signal=='NRZ':
        return decNRZ(seq,tb)
    elif signal=='NRZI':
        return decNRZI(seq,tb)
    elif signal=='Manchester':
        return decManchester(seq,tb)
    elif signal=='Manchester D':
        return decManchesterDiff(seq,tb)
    elif signal=='Miller':
        return decMiller(seq,tb)
    elif signal=='CMI':
        return decCMI(seq,tb)
    elif signal=='AMI':
        return decAMI(seq,tb)
    elif signal=='MLT3':
        return decMLT3(seq,tb)
    else:
        return dec2B1Q(seq,tb)